In [ ]:
# load libraries
import pandas as pd
import utils as ut
import importlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error



In [ ]:
# to have interactive html
import plotly.io as pio
pio.renderers.default = 'notebook'

In [ ]:
# read csv
df_consumption = pd.read_csv('consumption.csv')
df_costs = pd.read_csv('costs.csv')
df_net_imports = pd.read_csv('net_imports.csv')
df_price = pd.read_csv('price.csv')
df_production = pd.read_csv('production.csv')
df_weather = pd.read_csv('weather.csv')

In [ ]:
# Merge dataframes
df_merge = df_price.merge(df_weather, on='datetime',how='left')
df_merge = df_merge.merge(df_production,on='datetime',how='left')
df_merge = df_merge.merge(df_net_imports,left_on='datetime',right_on='Date',how='left')
df_merge = df_merge.merge(df_consumption,on='datetime',how='left')
df_merge['date'] = pd.to_datetime(df_merge['datetime']).dt.date
df_costs['date'] = pd.to_datetime(df_costs['date']).dt.date
df_merge = df_merge.merge(df_costs,on='date',how='left')
df_merge.drop(columns=['Date','date'],inplace=True)


### Clean Spark Spread
**Definition**: Measures profitability of electricity generated from natural gas after carbon costs.

$$
\text{Clean Spark Spread} = (\text{Electricity Price}) - (\text{Natural Gas Price} \times \text{Heat Rate}) - (\text{Carbon Price} \times \text{Emission Factor})
$$

### Clean Dark Spread
**Definition**: Measures profitability of electricity generated from coal after carbon costs.

$$
\text{Clean Dark Spread} = (\text{Electricity Price}) - (\text{Coal Price} \times \text{Heat Rate}) - (\text{Carbon Price} \times \text{Emission Factor})
$$

Where 
- Electricity Price: The market price of electricity.
- Coal/Gas Price: The price of coal/gas used as fuel.
- Heat Rate: The efficiency of the power plant, typically expressed in MWh per unit of fuel. A lower heat rate indicates higher efficiency.
- Carbon Price: The cost of carbon emissions per ton of CO₂.
- Emission Factor: The amount of CO₂ emitted per unit of coal consumed (typically in tons of CO₂ per MWh).

If Clean Spark Spread is significantly higher than Clean Dark Spread, then gas plants have advantage over coal plants.

# Initial Dataset Analysis

Before we go over the report, it's essential to notice that Clean Dark Spread (CDS) and Clean Spark Spread (CSS) are in daily format while other variables are in hourly granularity. It means that after merging, each daily value is repeated 24 times (24 hours per day).

Based on the profile report, we can notice the following things:

## Correlations

- There are variable pairs with relatively high linear correlations. E.g. CDS and HardCoal or CSS and Natural Gas. These correlations are expected to be due to the nature of the CDS and CSS explained before.
- Net Imports are highly negatively correlated with Wind behaviour. Germany's electricity generation relies heavily on renewables, especially wind. Thus, increased wind generation creates excess supply which Germany can export. These wind fluctuations could also affect price negative and positive spikes.
- The negative correlation of Wind Onshore (land generated) and Offshore (off-land generated) with gas-fired power is understandable. Wind has low systematic price, especially in comparison with Gas. So, with increase of cheap electricity from wind turbines, the profit margin of gas-fired power decreases.
- Consumption has positive correlation with gas and coal production, as with increase of consumption, plants start burning more gas and coal to match the demand.
- Interesting that the price has high correlation with Coal and CDS but not as high correlation with gas. Probably because coal has a much higher portion of generated electricity in German market (https://en.wikipedia.org/wiki/Electricity_sector_in_Germany).
- In relatively high correlations we can also notice such pairs as Temperature and Solar (with increase of sun, temperature also rises), Temperature and Dam and Ror (run-of-river), probably because of increased temperature there is melting snow, more rains than during winter and thus hydro generation grows.

## Variable Statistics

### Price:
- Highly volatile, with a range from -90.01 to 700 €/MWh
- Right-skewed distribution (mean 61.94, median 44.22)
- 2.1% of prices are negative, indicating occasional oversupply

### Wind Power (Onshore and Offshore):
- Onshore wind has a much higher mean production (11,299 MW) compared to offshore (2,730 MW) meaning that there are much more land turbines than turbines based on water.

### Solar:
- 42.7% of values are zero due to nighttime hours when it's not shining.

### Clean Spark Spread and Clean Dark Spread:
- Both datasets start from 2019.
- Clean Spark Spread is mostly negative (mean -12.45), suggesting gas plants often operate at a loss
- Clean Dark Spread is also mostly negative but less so (mean -5.67), indicating coal plants might be more competitive. Which could be surprising as gas plants should be more flexible at reacting to demand changes than coal. Still, it could be caused by the increased gas prices in 2021-2022.

### Net Import:
- Mean of -3,294 MW indicates Germany is a net exporter on average
- High variability, ranging from -17,807 MW (export) to 13,805 MW (import)

In the following cells we will see that volatility of Price, Clean Spark Spread and Clean Dark Spread is extremely affected from mid-2021 and up to the limit of the dataset due to Energy Crisis and War in Ukraine.

In [ ]:
from ydata_profiling import ProfileReport
profile1 = ProfileReport(df_merge, title="Power Data", explorative=True)

profile1.to_notebook_iframe()

# Exploratory Data Analysis and Feature Selection
From the report we also see that there are missing values in Wind. Let's have a look to these cases. There're 2 days where data is completely empty: 2018-12-31 and 2019-11-06. These days we will impute based on the previous day information. So, e.g. 2018-12-31 1:00 will receive data from 2018-12-30 1:00. 2019-11-07 00:00 will be backward imputed with 2019-11-07 01:00, the rest will be forward imputed. We assume that the nearest neighbors should have the best information about missing values. In addition, wind should also have seasonal patterns where usually it's lower during nights and higher during the day, thus we are imputing values based on previous day. 

In [ ]:
ut.missing_rows_nan(df_merge,'WIND').datetime.unique()

In [ ]:
%%capture
# Fill WIND values for 2018-12-31 using 2018-12-30 data
ut.fill_wind_from_previous_day(df_merge, '2018-12-31', '2018-12-30')

# Fill WIND values for 2019-11-06 using 2019-11-05 data
ut.fill_wind_from_previous_day(df_merge, '2019-11-06', '2019-11-05')

# Fill 2019-11-07 00:00 using 2019-11-07 01:00
df_merge['datetime'] = pd.to_datetime(df_merge['datetime'])
backfill_mask = (df_merge['datetime'] == pd.to_datetime('2019-11-07 00:00:00'))
impute_value = df_merge.loc[df_merge['datetime'] == pd.to_datetime('2019-11-07 01:00:00'), 'WIND'].values
if len(impute_value) > 0:
    df_merge.loc[backfill_mask, 'WIND'] = impute_value[0]

# Forward fill the remaining missing WIND values
df_merge['WIND'].fillna(method='ffill', inplace=True)

Let's visualize our time series and see their trajectories.
The task of this case study is to predict May and June 2022 of hourly da_prices, and the last data available is April 2022. da_prices were very stable until mid-2021 where they started growing and got into the highest volatility in Dec 2021-March 2022. April 2022 demonstrates a slight decrease in overall prices and the volatility suggesting that May and June 2022 could either stay at the same level as the second half of April or continue the slow down trend.

Variables Temperature, Wind, Dam, Solar, Ror, Wind OnShore and Offshore, Net Import and Consumption do not demonstrate and extreme spikes as these variables are not affected by geopolitics. So, we could for instance impute May and June values by using values of these variables in May and June from the past. We don't see any need in further transformations for these variables.

Nuclear shows a step-wise behaviour where January is usually the month of decrease in nuclear power production for a following year. We decided to use Nuclear production not in raw but after transformations.

CSS and CDS demonstrate similar volatility patterns, where these variables were stable until mid-2021 and after mid-2021 they increased their volatility. 

In [ ]:
_ = importlib.reload(ut)
df_merge.rename(columns={'datetime':'timestamp_rt'},inplace=True)
df_merge['timestamp_rt'] = pd.to_datetime(df_merge['timestamp_rt'])
column_list = ['price', 'TEMPERATURE', 'WIND', 'Dam', 'Solar',
       'NaturalGas', 'Ror', 'HardCoal', 'Nuclear', 'WindOnshore',
       'WindOffshore', 'NetImport', 'consumption', 'Clean Spark Spread',
       'Clean Dark Spread']
df_merge['node'] = 'Germany'

In [ ]:
ut.create_plot(df_merge,targets=column_list)

## Imputing
### Step 1. Consistent Variables 

Let's impute variables using their past observations. These variables had systematic seasonal trend, so we assume that imputation from the previous year could be our best assumption.

In [ ]:
df_merge.set_index(['timestamp_rt', 'node'], inplace=True)

# List of variables to impute
variables_to_impute = ['TEMPERATURE', 'WIND', 'Dam', 'Solar', 'NaturalGas', 'Ror', 'HardCoal', 'WindOnshore', 'WindOffshore', 'NetImport', 'consumption']

# Prepare the new data
new_data = ut.prepare_new_data(df_merge, variables_to_impute)

# Concatenate the new data to the original dataframe
df_merge = pd.concat([df_merge, new_data])

# Sort the index to ensure it's in chronological order
df_merge = df_merge.sort_index()

In [ ]:
# values are correctly imputed
print('2021-05-01 00:00:00')
df_merge.loc['2021-05-01 00:00:00']

In [ ]:
print('2022-05-01 00:00:00')
print('values are correctly imputed')
df_merge.loc['2022-05-01 00:00:00']

### Step 2. Nuclear Imputation

Step-wise Decrease in Nuclear: The Nuclear production exhibits a step-wise decrease during Januarys. It's visible that the last decrease happened in January 2022. 

The same way, as for other regressors, we will keep the logic assuming that data from the previous year (May-June 2021) is representative and can be used for imputing missing values for May-June 2022. However, we are not imputing the raw nuclear values but a percentage difference from the reference month - Median of January. We calculate Median January performance. Then, we calculate percentage changes with respect to the median. We then use percentages of May-June 2021 in May-June 2022 and multiply them by a reference median from January 2022. In other words, we assume that the production level is stable year to year but the nominal values are simply shifted down.

So, as an example, if a median production in January 2021 was 100MWh, and in May 2021 we see 120MWh, then it's 120%, which we move to May 2022 and multiply by the median of January 2022.

In [ ]:
# Function to get the median of January for Nuclear
def get_january_median(df, year):
    january_data = df.loc[(slice(f'{year}-01-01', f'{year}-01-31'), slice(None)), 'Nuclear']
    return january_data.median()

# Calculate the median January production for each year
january_medians = {year: get_january_median(df_merge, year) for year in range(2018, 2023)}

# Function to calculate the percentage of January median for each data point
def calculate_percentage(value, year):
    return value / january_medians[year] * 100

# Calculate the percentage values for Nuclear
df_merge['Nuclear_Percentage'] = df_merge.apply(lambda row: calculate_percentage(row['Nuclear'], row.name[0].year), axis=1)

# Use the percentage values from May-June 2021 to impute May-June 2022
may_june_2021 = df_merge.loc[(slice('2021-05-01', '2021-06-30'), slice(None)), 'Nuclear_Percentage']
may_june_2022 = may_june_2021.copy()
may_june_2022.index = pd.MultiIndex.from_arrays([may_june_2022.index.get_level_values(0) + pd.DateOffset(years=1), may_june_2022.index.get_level_values(1)])

df_merge.loc[(slice('2022-05-01', '2022-06-30'), slice(None)), 'Nuclear_Percentage'] = may_june_2022

# Convert the percentage back to absolute values for 2022
df_merge.loc[(slice('2022-05-01', '2022-06-30'), slice(None)), 'Nuclear'] = df_merge.loc[(slice('2022-05-01', '2022-06-30'), slice(None)), 'Nuclear_Percentage'] * january_medians[2022] / 100
df_merge.drop(columns=['Nuclear_Percentage'],inplace=True)



### Step 3. CSS and CDS

We need to deal with CSS and CDS. We can't simply impute them from 2021. These ratios are based on prices and thus are also extremely volatile. We tried different approaches and calculated different ratios and we believe that the best way would be to use rolling MA and STD for the difference between Price and CDS/CSS. This way we isolate:
$$
(\text{Commodity Price} \times \text{Heat Rate}) - (\text{Carbon Price} \times \text{Emission Factor}).
$$

Our strategy is to fit ARIMA models on historical data until the beginning of the changes in behavior Y2021. We then predict values for 2 months (data is in daily format, so we do 61 predictions). We subtract forecasted medians from values and add medians of April 2022. So, we keep historical level fluctuations but we shift the forecasts on the level of April 2022. 

We do it based on our assumption that May and June 2022 should keep similar historical movements but be on the similar levels as for April 2022. April demonstrates decreases in volatilities compared to March 2022, so it could be a good month for reference.

In [ ]:
df = df_merge.copy()

# first attempts
df['CDS_to_Price_Ratio'] = df['Clean Dark Spread'] / df['price']
df['CSS_to_Price_Ratio'] = df['Clean Spark Spread'] / df['price']
df['Normalized_Spread_Difference'] = (df['Clean Dark Spread'] - df['Clean Spark Spread']) / df['price']
window_2w = 24 * 7 * 2  # 2 week
window = 24 * 7  # 1 week
df['CDS_Rolling_Mean'] = df['Clean Dark Spread'].rolling(window_2w).mean()
df['CSS_Rolling_Mean'] = df['Clean Spark Spread'].rolling(window_2w).mean()
df['CDS_Rolling_Std1w'] = df['Clean Dark Spread'].rolling(window).std()
df['CSS_Rolling_Std1w'] = df['Clean Spark Spread'].rolling(window).std()
df['CDS_EMA'] = df['Clean Dark Spread'].ewm(span=24*7*2).mean()
df['CSS_EMA'] = df['Clean Spark Spread'].ewm(span=24*7*2).mean()

# Average Daily Price minus CDS/CSS
df_daily_avg = df.groupby([pd.Grouper(level='timestamp_rt', freq='D'), 'node']).mean()

# Calculate Price_minus_CDS and Price_minus_CSS
df_daily_avg['Price_minus_CDS'] = df_daily_avg['price'] - df_daily_avg['Clean Dark Spread']
df_daily_avg['Price_minus_CSS'] = df_daily_avg['price'] - df_daily_avg['Clean Spark Spread']
df_daily_avg.reset_index(inplace=True)

# Rename 'timestamp_rt' to 'date' for merging
df_daily_avg.rename(columns={'timestamp_rt': 'date'}, inplace=True)

df.reset_index(inplace=True)
df['date'] = pd.to_datetime(df['timestamp_rt']).dt.date
df_daily_avg['date'] = pd.to_datetime(df_daily_avg['date']).dt.date

# Merge the daily average back to the original DataFrame
df = df.merge(df_daily_avg[['date', 'node', 'Price_minus_CDS', 'Price_minus_CSS']], on=['date', 'node'], how='left')

# working options
df['Price_minus_CDS_MA1w'] = df['Price_minus_CDS'].rolling(window).mean()
df['Price_minus_CSS_MA1w'] = df['Price_minus_CSS'].rolling(window).mean()
df['Price_minus_CDS_SD1w'] = df['Price_minus_CDS'].rolling(window).std()
df['Price_minus_CSS_SD1w'] = df['Price_minus_CSS'].rolling(window).std()




In [ ]:
list_col = ['Price_minus_CSS','Price_minus_CDS','Price_minus_CDS_MA1w','Price_minus_CDS_SD1w','Price_minus_CSS_MA1w','Price_minus_CSS_SD1w']
ut.create_plot(df.reset_index(),targets=list_col)

In [ ]:
%%capture
df_merge.reset_index(inplace=True)
df_merge = df_merge.merge(df[['timestamp_rt', 'node','Price_minus_CDS_MA1w','Price_minus_CSS_MA1w', 'Price_minus_CDS_SD1w','Price_minus_CSS_SD1w']], on=['timestamp_rt', 'node'], how='left')
df_merge.drop(columns=['Clean Spark Spread', 'Clean Dark Spread'],inplace=True) # we drop them as we won't use them in May/June 2022 predictions


In [ ]:
%%capture
# Define the variables
variables = ['Price_minus_CDS_MA1w', 'Price_minus_CSS_MA1w', 'Price_minus_CDS_SD1w', 
             'Price_minus_CSS_SD1w']

# Define the dates
ma_train_end = '2021-01-31'
std_train_end = '2021-10-31'
forecast_start = '2022-05-01'
forecast_end = '2022-06-30'

# Create the 'date' column
df_merge['date'] = pd.to_datetime(df_merge['timestamp_rt']).dt.date

# Calculate the daily averages
df_daily_avg = df_merge.groupby('date')[variables].mean().reset_index()

# Process each variable one by one
for var in variables:
    if 'MA' in var:
        train_end = ma_train_end
    else:
        train_end = std_train_end
    
    adjusted_forecast = ut.predict_and_adjust(df_daily_avg.set_index('date')[var], train_end, forecast_start, forecast_end)
    
    # Add the forecast to the dataframe
    for date in adjusted_forecast.index:
        df_merge.loc[df_merge['date'] == date.date(), var] = adjusted_forecast.loc[date]


In [ ]:
list_col = ['Price_minus_CDS_MA1w','Price_minus_CDS_SD1w','Price_minus_CSS_MA1w','Price_minus_CSS_SD1w']
ut.create_plot(df_merge,targets=list_col)

### Step 4. Inspection of DA prices

We are visualizing weekly price profiles based on historical data up to 2021Y. We exclude more recent data because it's systematically different than the historical data and could corrupt the profiles.

As expected, working weeks have very similar two hill movements of prices during the day, when the minimum plateau is during early morning hours before everyone starts waking up and preparing for work. The peak hours are at 8 and 9AM. This is usually the time for spikes where demand is higher than supply. Then we have a decrease after 8AM, one more uphill when people start finishing their work and one more downmovement as everyone goes to sleep. Saturday and Sunday have absolutely different profiles, as most of the people don't work on these days. 

It suggests us to have a separate variable for day of the week and hours.

Unfortunately, as our task is to predict 2 months of prices during the shocking period, we can't use lags from e.g. last year prices because they are not relevant in 2022. What we can do is to enrich our X matrix with seasonal terms and try to model prices using our regressors.

In [ ]:
%matplotlib inline
ut.plot_weekly_profile(df_merge, 'Germany')

# Feature Generation and Backtesting
Let's now create additional features and prepare our matrix for regression. We use intraday, weekly and yearly fouerier terms and radial basis functions. We create categorical variables instead of dummies for hours, days of the week, months, years. We don't want to use dummies because we will be using lightgbm model, and it works fine with categorical variables.

In [ ]:
df_merge, fourier_cols = ut.create_fourier_terms(df_merge)

df_merge['hour'] = pd.Categorical(df_merge['timestamp_rt'].dt.hour)
df_merge = ut.create_RBF_features(df_merge, alpha=1000, col='hour') # 1000 is based on my previous work-experience. Evaluated it empirically.
df_merge['day_of_week'] = pd.Categorical(df_merge['timestamp_rt'].dt.dayofweek)
df_merge['month'] = pd.Categorical(df_merge['timestamp_rt'].dt.month)
df_merge['year'] = pd.Categorical(df_merge['timestamp_rt'].dt.year)

df_merge.set_index('timestamp_rt',inplace=True)
df_merge.drop(columns=['date','node'],inplace=True)

As we are limited in resources, we won't be able to tune hyperparameters sufficiently. LGBM is well-known for its well-performing default hyperparameters. We will try to tune them a bit using a small set of hyperparameter combinations. We will compare the best performing models based on RMSE, and we will check trajectories of our best model. We will fit lgbm on 1 year of data, test it on 2 months, then refit using sliding window. We will then compare final results up to April 2022 and then use the tuned final model for May and June 2022 predictions.


In [ ]:
%%capture
# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 100],
    'max_depth': [-1, 5, 10],
    'n_estimators': [100, 200, 500]
}

# Initialize the default parameters
default_params = {
    'objective': 'regression',
    'metric': 'rmse',
    "verbosity": -1,
    'boosting_type': 'gbdt',
    "num_threads": 2,
    'seed': 42
}

importlib.reload(ut)
# Define the backtesting period
predict_start = '2021-01-01'
predict_end = '2022-05-01' # not included

# Perform manual grid search
grid_search_results = ut.manual_grid_search(df_merge, param_grid, default_params, predict_start, predict_end)

# Print the results
for result in grid_search_results:
    print(f"Params: {result['params']}, RMSE: {result['rmse']}")

# Find the best parameters
best_result = min(grid_search_results, key=lambda x: x['rmse'])
print(f"Best parameters: {best_result['params']}, Best RMSE: {best_result['rmse']}")

In [ ]:
# Find the best parameters
best_result = min(grid_search_results, key=lambda x: x['rmse'])
print(f"Best parameters: {best_result['params']}, Best RMSE: {best_result['rmse']}")

Let's refit the best model and see its trajectories and feature importances

In [ ]:
%%capture
# Backtest with best parameters
best_params = best_result['params']
rmse_best, results_best, gain_importances_best, split_importances_best, models_best = ut.backtest_model(df_merge, best_params, predict_start, predict_end)
print('Best RMSE',rmse_best)

## Trajectories

Trajectories have a good seasonal power, they move similarly to actual values. Still, forecasts are often undervalued compared to actuals. But predictions perform pretty well during volatility times.

In [ ]:
results_best_visual = results_best.reset_index()
results_best_visual['node'] = 'Germany'
ut.create_plot(results_best_visual,targets=['actual','predicted'])

## Feature Importance

We now can evaluate feature importances. We will compare Shapley values with traditional methods based on gain and split importances.

SHAP measures the impact of variables taking into account the interaction with other variables.

Shapley values calculate the importance of a feature by comparing what a model predicts with and without the feature. 

However, since the order in which a model sees features can affect its predictions, this is done in every possible order,
so that the features are fairly compared.

This plot shows the 20 most important features of the best performing model. Features are ordered in descending order based on their importance. For each feature a distribution is plotted on how the train samples
influence the model outcome. The color corresponds to the values of the feature - the brighter is the red color, the higher is the feature value. If the color is blue, the feature values are low. X axis of the plot corresponds to the values of the target.

For Instance, HardCoal red values (high values of production generated from Hard Coal) correspond to high values for prices -> expected behavior. Coal is not a cheap resource compared to renewables, so it's profitable to burn it for electricity when prices are high. Another example, for instance, could be for wind -  high values of wind production correspond to low values for prices. Grey values for hour or month are due to the fact that these variables are categorical, so there's no high or low hour.

We can see that our Price-CDS/CSS features are also performing well, which should make our lgbm model to keep the forecast values of May and June 2022 at similar levels as it's in April 2022.

Shapley feature importances are more robust than standard ones based on split/gain importances. Still, it's always good to have a look to all three chart types and to see the matching variables. For instance, based on all three types of visualizations: 
- Naturalgas and Coal productions are one of the most important features for day-ahead price predictions. It's expected as these energy production resources take a big portion of all energy supply in Germany.
- Seasonal features such as hours, fourier terms also play important role, as they help to model systematic seasonal patterns.


In [ ]:
# Plot SHAP importances for the last model
ut.plot_shap_importances(models_best[-1], df_merge.drop(columns=['price']))

In [ ]:
_=ut.plot_importances(split_importances_best, gain_importances_best)

# May and June 2022 Prediction
We finally fit our best performing model on last year data up to 2022-04-30 23:00, and predict 2 months of data up to 22-06-30 23:00. We then can visualize trajectories and make a qualitative evaluation whether our predictions match our assumptions.

In [ ]:
%%capture
# Best hyperparameters
best_params = {'objective': 'regression', 'metric': 'rmse', 'verbosity': -1, 'boosting_type': 'gbdt', 'num_threads': 2, 'seed': 42, 'learning_rate': 0.1, 'num_leaves': 50, 'max_depth': 10, 'n_estimators': 500}

# Define the training and prediction periods
train_end_date = pd.to_datetime('2022-04-30 23:00:00')
predict_end_date = pd.to_datetime('2022-07-01 00:00:00')

# Fit the model and make predictions
predictions_df, best_model = ut.fit_and_predict(df_merge, best_params, train_end_date, predict_end_date)

# Save predictions to a CSV file
predictions_df.to_csv('predictions_may_june_2022.csv')

In [ ]:
df_final = df_merge.merge(predictions_df, on='timestamp_rt', how='left')
df_final['node']='Germany'

The output respects our assumption that May and June 2022 might have slightly lower but very similar behavior as April 2022. We see that day-ahead prices are slightly shifted down but respect the overall levels of April 2022.

In [ ]:
ut.create_plot(df_final.reset_index(),targets=['price','predicted'])